In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,057 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,774 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,253 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu foca

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
#@title
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [4]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('home_sales')

In [5]:
# View table
start_time = time.time()

spark.sql("""
SELECT * 
FROM home_sales
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date      |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-8283-4139a54d0d63|2022-01-08|2017      |424418|3       |2      

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

start_time = time.time()

spark.sql("""
SELECT 
DISTINCT date_built,
bedrooms,
round(mean(price), 2) as avgPrice
FROM home_sales 
WHERE bedrooms=4 
GROUP BY  1,2
ORDER BY date_built desc
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----------+--------+---------+
|date_built|bedrooms|avgPrice |
+----------+--------+---------+
|2017      |4       |296576.69|
|2016      |4       |296050.24|
|2015      |4       |307908.86|
|2014      |4       |299073.89|
|2013      |4       |299999.39|
|2012      |4       |298233.42|
|2011      |4       |302141.9 |
|2010      |4       |296800.75|
+----------+--------+---------+

--- 2.863520860671997 seconds ---


In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

start_time = time.time()

spark.sql("""
SELECT 
DISTINCT date_built,
bedrooms,
bathrooms,
round(mean(price), 2) as avgPrice
FROM home_sales 
WHERE bedrooms=3 AND bathrooms=3 
GROUP BY 1,2,3
ORDER BY date_built desc
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----------+--------+---------+---------+
|date_built|bedrooms|bathrooms|avgPrice |
+----------+--------+---------+---------+
|2017      |3       |3        |292676.79|
|2016      |3       |3        |290555.07|
|2015      |3       |3        |288770.3 |
|2014      |3       |3        |290852.27|
|2013      |3       |3        |295962.27|
|2012      |3       |3        |293683.19|
|2011      |3       |3        |291117.47|
|2010      |3       |3        |292859.62|
+----------+--------+---------+---------+

--- 1.349252700805664 seconds ---


In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

start_time = time.time()

spark.sql("""
SELECT 
DISTINCT date_built,
bedrooms,
bathrooms,
floors,
round(mean(price), 2) as avgPrice
FROM home_sales 
WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000
GROUP BY  1,2,3,4
ORDER BY date_built desc
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----------+--------+---------+------+---------+
|date_built|bedrooms|bathrooms|floors|avgPrice |
+----------+--------+---------+------+---------+
|2017      |3       |3        |2     |280317.58|
|2016      |3       |3        |2     |293965.1 |
|2015      |3       |3        |2     |297609.97|
|2014      |3       |3        |2     |298264.72|
|2013      |3       |3        |2     |303676.79|
|2012      |3       |3        |2     |307539.97|
|2011      |3       |3        |2     |276553.81|
|2010      |3       |3        |2     |285010.22|
+----------+--------+---------+------+---------+

--- 2.1387627124786377 seconds ---


In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT 
view,
round(avg(price), 2) as avgPrice
FROM home_sales 
GROUP BY view
HAVING avgPrice>=350000
ORDER BY view desc
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|avgPrice  |
+----+----------+
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
|80  |991767.38 |
+----+----------+
only showing top 20 rows

--- 1.2630457878112793 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT 
view,
round(avg(price), 2) as avgPrice
FROM home_sales 
GROUP BY view
HAVING avgPrice>=350000
ORDER BY view desc
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|avgPrice  |
+----+----------+
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
|80  |991767.38 |
+----+----------+
only showing top 20 rows

--- 0.5587842464447021 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [14]:
# 11. Read the parquet formatted data.
p_home_sales_df_p=spark.read.parquet('home_sales_partitioned')

In [15]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df_p.createOrReplaceTempView('p_home_sales_p')

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT 
view,
round(avg(price), 2) as avgPrice
FROM home_sales 
GROUP BY view
HAVING avgPrice>=350000
ORDER BY view desc
""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|avgPrice  |
+----+----------+
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
|80  |991767.38 |
+----+----------+
only showing top 20 rows

--- 0.8339171409606934 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")

all clear
